In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
from module.apps import App, PlotSim, PlotSim2, PlotSim3
import ipywidgets as widgets
import matplotlib.pyplot as plt
from module.utils import drias2020
from matplotlib.dates import DateFormatter

## Example: Impact assessment of Climate Change on the Selle watershed using a rainfall/runoff lumped-parameters model

This example corresponds to the "Selle et Morvillers" exercise as described in the Gardenia Tutorial manual (https://gardenia.brgm.fr). It aims at assessing the impacts of climate change on  the Selle watershed, using the DRIAS2020 regionalized climate projections (https://drias-climat.fr). These projections are produced by French climate modelling research institutes, based upon CMIP5 global climate models.

*Disclaimer: this example uses a modelling tool developed for the sole purpose of this exercise. It is based upon the functioning of the Gardenia model (https://gardenia.brgm.fr). Results shall not be interpreted nor used beyond the scope of this example. This online tool is not endorsed by BRGM nor does it encompasses all the capabilities of the Gardenia model.*

*Warning: This online tool uses the Binder executable environment (https://mybinder.org/). Binder automatically closes your session after 10 minutes of inactivity. If it appears that the tool does not work anymore, reload the webpage in a new tab in your web browser (https://mybinder.org/v2/gh/jpvergnes/formation_cc/master-english?urlpath=voila%2Frender%2Ftuto_hydro.ipynb). Note that if your browser uses an ad-blocker, some actions may be impeded. You may need to deactivate it for this exercise.*

The Selle river is a tributary of the Somme river located in northern France (Figure 1). Its watershed area covers about 524 km². Daily time-series of rainfall and potential evapo-transpiration (PET) averaged on 9 weather stations are available for this watershed. This dataset is illustrated on Figure 2. In addition, a daily time-serie of the Selle river flow from 1989 to 2003 is available at the gauging station "la Selle à Plachy"; as well as a monthly groundwater-level time-serie at the Morvillers monitoring well, from September 1985 to July 2003.

| <img src="bassin_selle.png" alt="Bassin de la Selle" width="500"/> |
|---|
| *Figure 1 : Location of the Selle watershed. The Morvillers monitoring well and the gauging station at Plachy ("la Selle à Plachy") are highlighted in red. The grid (8km square resolution) corresponds to the resolution of the DRIAS2020 regionalized climated projections* |

| <img src="pluie_etp.png" alt=""/> |
|---|
| *Figure 2 : Annual rainfall and PET* |


***
# Model calibration

The goal of this first session is to calibrate the model using the rainfall and PET data over the 1985-2003 period. The calibration (or history-matching) is achieved by modifying the various model parameters so as to match the observed flow and water-level data. Calibration performance is assessed visually, as well as by using statistical indicators, such as the Nash-Sutcliffe (Nash) and the Kling-Gupta-Efficicency (KGE) coefficients for the river flow, and the Nash coefficient for the water level.

To run the model, modify the parameter values using the user interface below. The simulated flow and groundwater level should vary accordingly, as well as the calibration criteria (e.g. KGE, Nash).

1. Carry out a sensitivity analysis on the calibration parameters (by modifying them in the interface) and discuss the impact of each parameter on the simulated flow and groundwater level.

2. Calibrate the model manually by modifying the parameters so as to maximise the calibration criteria (KGE, Nash) and minimize the residual between the observed and simulated time-series.

3. Write down the parameters you obtained through the manual calibration process. Now calibrate the model automatically by clicking the "Optimize" button. Compare the resulting parameters datasets.
***

In [2]:
pluie_obs = pd.read_csv(
    "Selle_et_Morvillers/Pluv_Moy_9_Stat_Selle_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0
)

etp_obs = pd.read_csv(
    "Selle_et_Morvillers/Etp_Moy_9_Stat_Selle_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0
)

debit = pd.read_csv(
    "Selle_et_Morvillers/Debit_Selle_Plachy_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0,
)['Debit']
debit = debit.replace(-2, np.nan)

niveau = pd.read_csv(
    "Selle_et_Morvillers/Niv_Morvillers_00608X0028_1985_2003.prn",
    delim_whitespace=True,
    parse_dates=True,
    dayfirst=True,
    index_col=0,
)['Niveau']
niveau = niveau.replace(9999., np.nan)

In [3]:
app = App(debit, niveau)
app.set_meteo(pluie_obs, etp_obs)
column1 = widgets.VBox(
    [
        widgets.Label("Parameters", style=dict(font_weight="bold", background='lightblue')),
        app.widgetA,
        app.widgetR,
        app.widgetTHG,
        app.widgetTG1,
        app.widgetbaseNiv,
        app.widgetcoeffEmmag,
        widgets.Label("Criteria", style=dict(font_weight="bold", background='lightblue')),
        app.textNash,
        app.textKGE,
        app.textNashNiv
    ],
    layout=widgets.Layout(width='23%')
)

column2 = widgets.VBox(
    [
        app.btnOpti,
        app.fig.canvas,
    ]
)

widgets.HBox([column1, column2])

***
# Climate change impact assessment

The goal of this following session is to use the the DRIAS2020 regionalized climate projections (https://drias-climat.fr) to forecast the Selle river flow at Plachy and the groundwater level at Morvillers using the calibrated model.

## Forecasting

For the purpose of this exercise, the DRIAS2020 dataset have already been downloaded and pre-processed in order ot be used directly with the model. This dataset corresponds to daily rainfall and PET time-series from 12 ensembles of global climate models (GCM) and regional climate models (RCM) over the historical period (from 1950 to 2005) and the future (from 2005 to 2100) for each representative concentration pathway (RCP) scenario (using the IPCC 5th report definition).
For each GCM/RCM ensemble, the pre-processing included:
-  Identifying the DRIAS2020 grid cells corresponding to the study area (e.g. the 8km square cells superimposing on the Selle watershed in Figure 1);
- Calculating a spatial weighted average of the rainfall and PET at each time-step, in order to aggregate all the available time-series over the watershed. This weighted average is carried out according to the relative watershed area within the DRIAS2020  cell-size. Two aggregated rainfall and PET time-series are eventually obtained, that we will use in the model.
- The above process is repeated for each GCM/RCM ensemble and RCPs so that 30 projections of rainfall/PET from 1950 to 2100 are available as inputs for the model.

Run these simulations by clicking the "Run" button below. The calibrated model is now calculating the forecast river flow and groundwater-level using the 30 rainfall/PET projections. Model runs are completed when the progress bar is full. Let's move on to the next session to analyse the results.

In [4]:
def run_simulations(b):
    btn.disabled = True
    progress.value = 0
    for gcm, rcm, period in drias2020:
        pluie = 'sims_cc/{0}_{1}_{2}/Pluie_Selle_et_Morvillers_{0}_{1}_{2}'.format(
            gcm,
            rcm,
            period
        )
        pluie = pd.read_csv(
            pluie,
            delim_whitespace=True,
            parse_dates=True,
            dayfirst=True,
            index_col=0
        )
        etp = 'sims_cc/{0}_{1}_{2}/ETP_Selle_et_Morvillers_{0}_{1}_{2}'.format(
            gcm,
            rcm,
            period
        )
        etp = pd.read_csv(
            etp,
            delim_whitespace=True,
            parse_dates=True,
            dayfirst=True,
            index_col=0
        )
        app.set_meteo(pluie, etp)
        app.run(-9999)
        os.makedirs('results_cc', exist_ok=True)
        app.result.to_csv(
            'results_cc/{0}_{1}_{2}.csv'.format(
                gcm, rcm, period
            ),
        )
        progress.value += 1
    app.set_meteo(pluie_obs, etp_obs)
    btn.disabled = False
    progress.value = 0
btn = widgets.Button(
    description='Run',
    disabled=False,
    icon='check'
)
btn.on_click(run_simulations)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=len(drias2020),
    bar_style='info', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
widgets.HBox([btn, progress])

***
## Results
### Time-series plots

Simulated time-series of river flow and groundwater level can be displayed in the interface below, using several representations:

- Select the GCM/RCM ensemble of your choice in the "Models" section. Multiple selection is possible with your mouse by maintaining the "Ctrl" key.
- Select the RCP of your choice in the "RCPs" section. Multiple selection is possible with your mouse by maintaining the "Ctrl" key.
- Select the model output of your choice in the "Output" section, i.e. either the Selle river flow at Plachy or the groundwater level at Morvillers
- Select a variable to display in the "Variable" section, i.e. the daily output or the minimum/mean/maximum monthly or annual mean;
- Select a representation mode in the "Representation" section, as follow:
    - in the 'Spaghetti' mode, all model ensembles are plotted on the same plot;
    - in the 'Q5/median/Q95' mode, an envelope of all the model ensembles is plotted, representing respectively the 5% quantile, the median and the 95% quantile based on all the selected models population; 
    - Similarly the 'Min/Mean/Max' plots the model ensembles envelope using the minimum, the mean and the maximum of all the models.
- Check the "Anomaly" box to display the results in terms of anomaly relative to the annual mean over the 1976-2005 reference period.


### Mann-Kendall statistical test for monotonic trend

If the selected variable is annual (min, mean or max), a Mann-Kendall test is performed on the simulated time-serie, which results are displayed below the graph. 
For reference, this test is carried out using the Python package "pymannkendall" (https://pypi.org/project/pymannkendall/). The test indicates :
- If a trend is detected ("decreasing", "increasing", "no-trend");
- The significance of the test is calculated through the p-value (i.e. null-hypothesis testing). If the p-value is below the threshold value of 5% (0.05), the null-hypothesis test can be rejected and the trend is very likely. p-values higher than this threshold suggests that there is no trend.

### A few questions

| <img src="donnes_modeles.png" alt="" width=600/> | <img src="selection_drias2020.png" alt="" width=400/> |
|---|---|
| *Figure 3 : GCM/RCM ensembles spread toward 2100 for RCP8.5, mean over France* | *Figure 4 : Available GCM/RCM ensembles in the DRIAS2020 dataset* |

The Figure 3 above highlights the behavior of the DRIAS2020 projections related to their increase in temperature (y-axis) and their change in precipitations, for the forecast period 2070-2100, averaged over France. The median of the models suggest an increase in the precipitations over France, as well as a 3.9°C increase in temperature for this period. Figure 4 indicates all the GCM/RCM/RCP ensembles available in DRIAS2020.

1. On Figure 3, what is the most humid GCM/RCM ensemble? the driest?
2. Select the driest GCM/RCM and one or several RCPs, and try out some of the possible representation modes. When selecting the annual variables, what are the trends suggested by the model?
3. Repeat the same process with the most humid GCM/RCM combination. When selecting the annual variables, what are the trends suggested by the model?
4. Select all the GCM/RCM ensembles and try out the representation modes 'Spaghetti', 'Q5/Median/Q95' and 'Min/Mean/Max'. For the latter two representations, what are the trends suggested by the model?
5. How does the model uncertainty vary with time? Which RCP features the most uncertainty?
6. In light of Figure 2, if you had to choose 5 representative GCM/RCM ensembles out of the 12 available, which ones would you select? Assess the simulated trends for this selection and compare them to those estimated in the previous questions.

***

In [5]:
plot = PlotSim()
widgets.VBox([plot.main, plot.label, plot.out])

***
### Seasonal variability analysis

For this session, we reiterate the previous analysis by assessing the daily or monthly seasonal behavior of the projections over the historical reference period (1976-2005) and three future periods 2021-2050, 2041-2070 and 2071-2100. Anomalies are calculated relative to the corresponding seasonal cycle over the historical reference period. In addition, interannual mean values are indicated below the graph.

Try out the different representation modes. Answer the above questions using a given RCP, a given period of analysis, etc.

***

In [6]:
plot2 = PlotSim2()
widgets.VBox([plot2.main, plot2.label, plot2.out])

***
## Appendix: changes in rainfall and PET
***

In [7]:
plot3 = PlotSim3()
widgets.VBox([plot3.main, plot3.label, plot3.out])